In [ ]:
from rechunker import rechunk
import s3fs
#import xarray as xr
import zarr
import dask.array as dsa
import shutil
from dask.diagnostics import ProgressBar
import numpy as np
import xarray as xr
import os
import fsspec
import pandas as pd

In [ ]:
#from dask_gateway import Gateway
#from dask.distributed import Client

In [ ]:
#gateway = Gateway()
#cluster = gateway.new_cluster(worker_memory=8)
#cluster.adapt(minimum=1, maximum=60)
#client = Client(cluster)
#cluster

In [ ]:
file_aws = 'https://mur-sst.s3.us-west-2.amazonaws.com/zarr-v1'
file_aws2 = 'https://mur-sst.s3.us-west-2.amazonaws.com/zarr'
fout3 = 'data/test_subset_v2.zarr'
rechunk_location = 'noaa-oisstv02r1/newgroupv2.zarr'
tmp_location = 'noaa-oisstv02r1/tmpv2.zarr'

# create a subset of the data to test rechunker
ds_aws = xr.open_zarr(file_aws,consolidated=True) #read in data
ds_aws = ds_aws.isel(time=slice(0,20),lat=slice(6000,7000),lon=slice(19000,20000)) #subset to reasonable size
ds_aws.to_zarr(fout3,consolidated=True)  #output data
xlat,xlon = -26,18
print('v1',ds_aws.analysed_sst.sel(lat=xlat,lon=xlon).isel(time=0).compute().data)

In [ ]:
# load the data
ds_zarr = zarr.open_consolidated(fout3, mode='r')
print(zarr.tree(ds_zarr))

In [ ]:
#rechunker plan
#s3 = s3fs.S3FileSystem(client_kwargs=dict(region_name='us-east-2'), default_fill_cache=False, skip_instance_cache=True)

s3 = s3fs.S3FileSystem(client_kwargs=dict(region_name='us-west-2'),anon=False, default_fill_cache=False, skip_instance_cache=True)

s3_rechunk_store = s3fs.S3Map(root=rechunk_location, create=True, s3=s3)

# Note this path must exist in S3 or will raise rechunker assertion, `assert temp_store_or_group is not None`
s3_tmp_store     = s3fs.S3Map(root=tmp_location, create=True, s3=s3)


In [ ]:
target_chunks = {
    'analysed_sst': {'time': 20, 'lat': 100, 'lon': 100},
    'analysis_error': {'time': 20, 'lat': 100, 'lon': 100},
    'mask': {'time': 20, 'lat': 100, 'lon': 100},
    'sea_ice_fraction': {'time': 20, 'lat': 100, 'lon': 100},
    'lat': None,
    'lon': None,
    'time': None
}
max_mem = '2GB'

array_plan = rechunk(ds_zarr, target_chunks, max_mem, s3_rechunk_store, s3_tmp_store)
array_plan

In [ ]:
## CHECK filename here DOES This have nan or fill_value?
ds_aws = xr.open_zarr(s3_rechunk_store,consolidated=True) #read in data
xlat,xlon = -26,18
print('v1',ds_aws.analysed_sst.sel(lat=xlat,lon=xlon).isel(time=0).compute().data)